In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Read hurricane CSV in for further data filtering and cleansing
hurricane_df = pd.read_csv('../01_Extract/Output/hurricane_df.csv')
hurricane_df.head()

,Rank,#,Date,Time,Latitude,Longitude,Max Winds(kt),SSHWS,RMWnm,CentralPressure(mb),States Affected,Name
0,Rank,#,Date,Time,Latitude,Longitude,Max Winds(kt),SSHWS,RMWnm,CentralPressure(mb),States Affected,Name
1,1,3,9/3/1935,0200Z,24.8N,80.8W,160,5,5,892,"CFL5,BFL5","""Labor Day"""
2,2,9,8/18/1969,0400Z,30.3N,89.4W,150,5,10,900,"MS5,LA5,AL1",Camille
3,3,4,8/26/1992,0905Z,25.5N,80.3W,145,5,10,922,"CFL5,BFL4",Andrew
4,4,14,10/10/2018,1730Z,30.0N,85.5W,140,5,10,919,"AFL5,I-GA2",Michael


In [3]:
# Remove first row with duplicate table headers
hurricane_df = hurricane_df.iloc[1:]

In [4]:
# Remove columns not required for targeted analysis
hurricane_df = hurricane_df.drop(['Rank', '#'], axis=1)
hurricane_df.head()

,Date,Time,Latitude,Longitude,Max Winds(kt),SSHWS,RMWnm,CentralPressure(mb),States Affected,Name
1,9/3/1935,0200Z,24.8N,80.8W,160,5,5,892,"CFL5,BFL5","""Labor Day"""
2,8/18/1969,0400Z,30.3N,89.4W,150,5,10,900,"MS5,LA5,AL1",Camille
3,8/26/1992,0905Z,25.5N,80.3W,145,5,10,922,"CFL5,BFL4",Andrew
4,10/10/2018,1730Z,30.0N,85.5W,140,5,10,919,"AFL5,I-GA2",Michael
5,8/10/1856$,1800Z,29.2N,91.1W,130,4,10,934,LA4,"""Last Island"""


In [5]:
# Review storm name values
hurricane_df['Name'].values

array(['"Labor Day"', 'Camille', 'Andrew', 'Michael', '"Last Island"',
       '"Indianola"', '---------', '"Freeport"', 'Charley', 'Laura',
       'Ida', 'Ian', '"Great Miami"', '"Lake Okeechobee"', 'Donna',
       'Carla', '"Galveston"', 'Celia', 'Hugo', '"Chenier Caminanda"',
       '----------', '"Galveston"', '----------', '----------',
       '----------', '----------', '----------', 'King', 'Hazel',
       'Gracie', 'Betsy', 'Harvey', 'Irma', '"Middle Gulf Shore"',
       '----------', '----------', '----------', '----------',
       '----------', '----------', '"New Orleans"', '----------',
       '----------', '----------', 'Audrey', 'Helene', 'Katrina',
       '---------', '---------', '---------', '---------', '---------',
       '---------', '---------', '"Great New England"', '---------',
       'Easy', 'Ivan', 'Jeanne', 'Dennis', 'Wilma',
       '"Great Middle Florida"', '"Great Mobile"', '"Great Carolina"',
       'Betsy', '"Eastern New England"', '----------', '---------

In [6]:
# Remove unnamed storm values
hurricane_df = hurricane_df[~hurricane_df['Name'].isin(['---------', '----------'])]

In [7]:
# Confirm storm values are only named storms
hurricane_df['Name'].values

array(['"Labor Day"', 'Camille', 'Andrew', 'Michael', '"Last Island"',
       '"Indianola"', '"Freeport"', 'Charley', 'Laura', 'Ida', 'Ian',
       '"Great Miami"', '"Lake Okeechobee"', 'Donna', 'Carla',
       '"Galveston"', 'Celia', 'Hugo', '"Chenier Caminanda"',
       '"Galveston"', 'King', 'Hazel', 'Gracie', 'Betsy', 'Harvey',
       'Irma', '"Middle Gulf Shore"', '"New Orleans"', 'Audrey', 'Helene',
       'Katrina', '"Great New England"', 'Easy', 'Ivan', 'Jeanne',
       'Dennis', 'Wilma', '"Great Middle Florida"', '"Great Mobile"',
       '"Great Carolina"', 'Betsy', '"Eastern New England"',
       '"Sea Island"', '"Velasco"', '"Grand Isle"', '"Tampa Bay"',
       'Carol', 'Alma', 'Beulah', 'Alicia', 'Elena', 'Andrew', 'Emily',
       'Opal', 'Fran', 'Bret', 'Rita', 'Zeta', 'Idalia'], dtype=object)

In [8]:
# Remove symbol codes stored within the date field
hurricane_df['Date'] = hurricane_df['Date'].replace({'[$*#]': ''}, regex=True)

In [9]:
# Clean date in order to format
def clean_date(date_str):
    if isinstance(date_str, str):
        # Remove leading slashes and spaces
        date_str = date_str.replace('', '')
        date_str = date_str.strip().lstrip('/')
        return date_str
    return date_str

hurricane_df['Date'] = hurricane_df['Date'].apply(clean_date)

In [10]:
# Convert Date to date format
hurricane_df['Date'] = pd.to_datetime(hurricane_df['Date'], format='%m/%d/%Y')

In [11]:
# Convert 'HHMMZ' format to normal time format
def convert_time(time_str):
    if isinstance(time_str, str) and time_str.endswith('Z'):
        # Remove the 'Z' and parse the time
        time_str = time_str[:-1]
        # Parse the time string into a datetime object and change to 12-hour format with AM/PM
        time_obj = datetime.strptime(time_str, '%H%M')
        return time_obj.strftime('%I:%M %p')
    return time_str

# Apply the conversion function
hurricane_df['Time'] = hurricane_df['Time'].apply(convert_time)

In [12]:
# Convert latitude and longitude for future mapping
def convert_coordinates(coord_str):
    if isinstance(coord_str, str):
        # Remove the directional character (N/S/E/W)
        direction = coord_str[-1]
        value = float(coord_str[:-1])
        # Adjust the sign based on the direction
        if direction in ['S', 'W']:
            value = -value
        return value
    return None

# Apply the conversion function
hurricane_df['Latitude'] = hurricane_df['Latitude'].apply(convert_coordinates)
hurricane_df['Longitude'] = hurricane_df['Longitude'].apply(convert_coordinates)

In [13]:
# Convert number columns to numeric values
hurricane_df['Max Winds(kt)'] = pd.to_numeric(hurricane_df['Max Winds(kt)'], errors='coerce')
hurricane_df['SSHWS'] = pd.to_numeric(hurricane_df['SSHWS'], errors='coerce')
hurricane_df['RMWnm'] = pd.to_numeric(hurricane_df['RMWnm'], errors='coerce')
hurricane_df['CentralPressure(mb)'] = pd.to_numeric(hurricane_df['CentralPressure(mb)'], errors='coerce')

In [14]:
# Review dataframe
hurricane_df.head(10)

,Date,Time,Latitude,Longitude,Max Winds(kt),SSHWS,RMWnm,CentralPressure(mb),States Affected,Name
1,1935-09-03,02:00 AM,24.8,-80.8,160,5,5.0,892,"CFL5,BFL5","""Labor Day"""
2,1969-08-18,04:00 AM,30.3,-89.4,150,5,10.0,900,"MS5,LA5,AL1",Camille
3,1992-08-26,09:05 AM,25.5,-80.3,145,5,10.0,922,"CFL5,BFL4",Andrew
4,2018-10-10,05:30 PM,30.0,-85.5,140,5,10.0,919,"AFL5,I-GA2",Michael
5,1856-08-10,06:00 PM,29.2,-91.1,130,4,10.0,934,LA4,"""Last Island"""
6,1886-08-20,01:00 PM,28.1,-96.8,130,4,15.0,925,BTX4,"""Indianola"""
8,1932-08-14,04:00 AM,29.0,-95.2,130,4,10.0,935,"CTX4,BTX1","""Freeport"""
9,2004-08-13,07:45 PM,26.6,-82.2,130,4,5.0,941,"BFL4,CFL1,DFL1",Charley
10,2020-08-27,06:00 AM,29.8,-93.3,130,4,15.0,939,"LA4,ATX1",Laura
11,2021-08-29,04:55 PM,29.1,-90.2,130,4,10.0,931,LA4,Ida


In [15]:
# Review columns types
hurricane_df.dtypes

Date                   datetime64[ns]
Time                           object
Latitude                      float64
Longitude                     float64
Max Winds(kt)                   int64
SSHWS                           int64
RMWnm                         float64
CentralPressure(mb)             int64
States Affected                object
Name                           object
dtype: object

In [16]:
# Sort hurricanes based on date, reset index
hurricane_df = hurricane_df.sort_values(by='Date')
cleaned_hurricane_df = hurricane_df.reset_index(drop=True)
cleaned_hurricane_df

,Date,Time,Latitude,Longitude,Max Winds(kt),SSHWS,RMWnm,CentralPressure(mb),States Affected,Name
0,1851-08-23,09:00 PM,30.1,-85.7,100,3,NaN,955,"AFL3,GA1","""Great Middle Florida"""
1,1852-08-26,06:00 AM,30.2,-88.6,100,3,10.0,961,"AL3,MS3,LA2,AFL1","""Great Mobile"""
2,1854-09-08,08:00 PM,31.7,-81.1,100,3,40.0,950,"GA3,SC2,DFL1","""Great Carolina"""
3,1855-09-16,03:00 AM,29.2,-89.5,110,3,NaN,945,"LA3,MS3","""Middle Gulf Shore"""
4,1856-08-10,06:00 PM,29.2,-91.1,130,4,10.0,934,LA4,"""Last Island"""
5,1869-09-08,10:00 PM,41.4,-71.7,100,3,30.0,965,"RI3,MA3,CT1","""Eastern New England"""
6,1886-08-20,01:00 PM,28.1,-96.8,130,4,15.0,925,BTX4,"""Indianola"""
7,1893-08-28,05:00 AM,31.7,-81.1,100,3,25.0,954,"GA3,SC3,NC1,DFL1","""Sea Island"""
8,1893-10-02,08:00 AM,29.3,-89.8,115,4,10.0,948,LA4,"""Chenier Caminanda"""
9,1900-09-09,02:00 AM,29.1,-95.1,120,4,15.0,936,CTX4,"""Galveston"""


In [17]:
# Save cleaned huricane data for future use
cleaned_hurricane_df.to_csv('../02_Transform/Output/cleaned_hurricane_df.csv',index=False)